In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import os
from urllib import parse
import numpy as np

네이버 영화 메인 페이지 1~10위 영화 상세페이지 주소 가져오기

In [2]:
def step1_get_detail_url() :
    # 접속할 페이지의 주소
    site = 'https://movie.naver.com/movie/running/current.nhn?order=reserve'

    # 접속한다.
    response = requests.get(site)   

    bs = BeautifulSoup(response.content, 'html.parser')
    #print(bs)

    # a 태그들을 가져온다.
    a_list = bs.select('.top_thumb_lst a')

    # href 속성을 가져온다.
    df = pd.DataFrame()
    for idx in range(10) :       # 상위 10개만 가져오기
        href = a_list[idx].get('href')
        
        # 가져온 href 속성의 주소를 분석한 객체를 생성한다.
        a1 = parse.urlparse(href)
        
        # 주소를 분석한 객체서 쿼리 스트링을 가져온다
        query_str = parse.parse_qs(a1.query)
        
        # 추출한 쿼리스트링 데이터에서 원하는 파라미터 데이터를 추출한다.
        code = query_str['code'][0]
        

        # print(href)
        df = df.append([[code]], ignore_index=True)

    df.columns = ['code']
    df.to_csv('movie_code_list.csv', index=False, encoding='utf-8-sig')
    print('주소 저장 완료')

In [ ]:
step1_get_detail_url()

평점 더보기 주소 스크랩

In [3]:
def step2_get_reple_href() :
    # 스크랩한 영화 코드를 불러온다.
    code_frame = pd.read_csv('movie_code_list.csv')
    code_list = code_frame['code'].tolist()

    # 테스트용
 #   code_list = ['156464', '109906']


    # 영화코드와 주소를 합쳐서 요청할 주소를 만든다.
    url_list = pd.DataFrame()
    for code in code_list:
        site = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=%s&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false' % code
        # print(site)
        url_list = url_list.append([[site]], ignore_index=True)

    url_list.columns = ['url']
    url_list.to_csv('movie_url_list.csv', index=False, encoding='utf-8-sig')
    print('저장완료')

In [ ]:
step2_get_reple_href()

예매순 1~10 영화의 1페이지부터 마지막페이지 까지 순회하면서 평점과 리뷰 데이터 가져와서 저장

In [4]:
def step3_get_reply_data() :
    # csv에 저장되어 있는 url 데이터를 가져온다.
    df = pd.read_csv('movie_url_list.csv')
    url_list = df['url'].tolist()

    # for url in url_list :
    #    print(url)
    # url = url_list[0]

    for url in url_list :
        print(url)
        # 해당 영화의 첫 페이지 html 데이터를 가져온다. (총 몇건의 리뷰가 있는지 확인해서 "페이지수"를 계산하기 위함)
        response = requests.get(url)
        bs = BeautifulSoup(response.content, 'html.parser')
        # print(bs)

        # 총 페이지 수를 구한다.
        strong = bs.select('.total em')
        #1->0
        score_total = int(strong[0].text.replace(',', ''))    # 쉼표 없애기 / int(정수형)로 만들기
        # print(score_total)
        pageCnt = score_total // 10     # 한페이지당 10개의 리뷰가 있어서
        if score_total % 10 > 0 :
            pageCnt += 1
        print(pageCnt)

        # 전체 페이지를 돌면서 140평 데이터를 가져온다.
        # 현재 페이지
        now_page = 1

        #pageCnt = 5        # 테스트로 5페이지까지만
        
        while now_page <= pageCnt :
            #sleep(1)
            #sleep(0.0001)
            # 요청할 페이지의 주소
            url2 = url + '&page=' + str(now_page)
            # print(url2)

            # 140자평 데이터를 추출한다.
            response2 = requests.get(url2)
            bs2 = BeautifulSoup(response2.content, 'html.parser')

            result_df = pd.DataFrame()

            # li 태그들을 가져온다.(score_reple 태그-리뷰-를 포함하고 있는)
            lis = bs2.select('.score_result li')

            for obj in lis :
                # 평점
                star_score = obj.select('.star_score em')[0].text
                # 140자평
                score_reple = obj.select('.score_reple p')[0].text
                # print(star_score)
                # print(score_reple)

                # 저장한다.
                result_df = result_df.append([[score_reple, star_score]], ignore_index=True)

            if os.path.exists('star_score.csv') == False :     # 아직 파일이 없으면 파일을 만든다.
                result_df.columns = ['text', 'score']
                result_df.to_csv('star_score.csv', index=False, encoding='utf-8-sig')
            else :                                               # 이미 파일이 있으면 결과를 더한다.
                result_df.to_csv('star_score.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

            print("%d / %d" % (now_page, pageCnt))               # 진행경과를 보여준다. n번째 중 몇 번째 진행중인지.
            now_page += 1

    print('저장완료')

In [ ]:
step3_get_reply_data()

In [5]:
# 140자평 데이터 전처리 함수
def text_preprocessing(text) :
    if text.startswith('관람객') :
        return text[3:]
    ######
    elif text.startswith('스포일러가 포함된 감상평입니다. 감상평 보기'):
        return text[23:]
    #####
    else :
        return text

# 평점 전처리 함수
def star_preprocessing(text) :
    value = int(text)

    if value <= 5 :
        return '0'
    else :
        return '1'

In [6]:
def step4_data_preprocessing() :
    # 수집한 데이터를 읽어온다.
    df = pd.read_csv('star_score.csv')

    # 전처리를 수행한다.
    df['text'] = df['text'].apply(text_preprocessing)
    df['score'] = df['score'].apply(star_preprocessing)

    # 학습데이터와 테스트 데이터로 나눈다.
    text_list = df['text'].tolist()
    star_list = df['score'].tolist()

    from sklearn.model_selection import train_test_split

    # 70%는 학습, 30%는 test
    text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size=0.3, random_state=0)

    return text_train, text_test, star_train, star_test


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle
from konlpy.tag import *

In [7]:
# 형태소 분석을 위한 함수
def tokenizer(text) :
    okt = Okt()
    return okt.morphs(text)

In [8]:
def step5_learning(X_train, y_train, X_test, y_test):
    # 주어진 데이터를 단어 사전으로 만들고 각 단어의 빈도수를 계산한 후 벡터화 하는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)

    # 문장별 나오는 단어수 세서 수치화, 벡터화해서 학습을 시킨다.
    logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)

    pipe = Pipeline([('vect', tfidf), ('clf', logistic)])

    # 학습한다.
    pipe.fit(X_train, y_train)

    # 학습 정확도 측정
    y_pred = pipe.predict(X_test)
    print(accuracy_score(y_test, y_pred))

    # 학습된 모델을 저장한다.
    with open('pipe.dat', 'wb') as fp :
        pickle.dump(pipe, fp)
        
    print('저장완료')

In [9]:
def step6_using_model() :
    # 객체를 복원한다.
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)

    import numpy as np

    while True :
        text = input('리뷰를 작성해주세요 :')

        str = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(str) * 100)
        # 예측 결과
        r2 = pipe.predict(str)[0]

        if r2 == '1' :
            print('긍정적인 리뷰')
        else :
            print('부정적인 리뷰')

        print('정확도 : %.3f' % r1)

In [10]:
def one_movie_get_reply_data() :
    # csv에 저장되어 있는 url 데이터를 가져온다.
    df = pd.read_csv('movie_url_list.csv')
    url_list = df['url'].tolist()

    # for url in url_list :
    #    print(url)
    url = url_list[0]

       
    print(url)
    # 해당 영화의 첫 페이지 html 데이터를 가져온다. (총 몇건의 리뷰가 있는지 확인해서 "페이지수"를 계산하기 위함)
    response = requests.get(url)
    bs = BeautifulSoup(response.content, 'html.parser')
    # print(bs)

    # 총 페이지 수를 구한다.
    strong = bs.select('.total em')
    #1->0
    score_total = int(strong[0].text.replace(',', ''))    # 쉼표 없애기 / int(정수형)로 만들기
    # print(score_total)
    pageCnt = score_total // 10     # 한페이지당 10개의 리뷰가 있어서
    if score_total % 10 > 0 :
        pageCnt += 1
    print(pageCnt)

    # 전체 페이지를 돌면서 140평 데이터를 가져온다.
    # 현재 페이지
    now_page = 1

#          pageCnt = 5        # 일단 테스트로 5페이지까지만 해보자.

    while now_page <= pageCnt :
            #sleep(1)
            #sleep(0.0001)
            # 요청할 페이지의 주소
            url2 = url + '&page=' + str(now_page)
            # print(url2)

            # 140자평 데이터를 추출한다.
            response2 = requests.get(url2)
            bs2 = BeautifulSoup(response2.content, 'html.parser')

            result_df = pd.DataFrame()

            # li 태그들을 가져온다.(score_reple 태그-리뷰-를 포함하고 있는)
            lis = bs2.select('.score_result li')

            for obj in lis :
                # 평점
                star_score = obj.select('.star_score em')[0].text
                # 140자평
                score_reple = obj.select('.score_reple p')[0].text
                # print(star_score)
                # print(score_reple)

                # 저장한다.
                result_df = result_df.append([[score_reple, star_score]], ignore_index=True)

                if os.path.exists('1917.csv') == False :     # 아직 파일이 없으면 파일을 만든다.
                    result_df.columns = ['text', 'score']
                    result_df.to_csv('1917.csv', index=False, encoding='utf-8-sig')
                else :                                               # 이미 파일이 있으면 결과를 더한다.
                    result_df.to_csv('1917.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

            print("%d / %d" % (now_page, pageCnt))               # 진행경과를 보여준다. n번째 중 몇 번째 진행중인지.
            now_page += 1


    print('저장완료')

In [11]:
# 스크래핑 함수
def scrapping() :

    # 각 영화 코드 데이터를 가져와 저장한다.
    step1_get_detail_url()

    # 140자 평 데이터가 있는 페이지의 주소를 저장한다.
    step2_get_reple_href()

    # 140평 데이터를 가져온다.
    step3_get_reply_data()

# 학습 함수
def learing() :
    text_train, text_test, star_train, star_test = step4_data_preprocessing()
    step5_learning(text_train, star_train, text_test, star_test)

# 사용 함수
def using() :
    step6_using_model()

In [ ]:
scrapping()

In [12]:
import pandas as pd

In [ ]:
using()

In [ ]:
# scrapping()
learing()
#using()

In [13]:
def get_data(url,re_list):
    rslist = []
    count=0
    resp = requests.get(url)
    html = BeautifulSoup(resp.content, 'html.parser')
    score_result = html.find('div', {'class': 'score_result'})
    lis = score_result.findAll('li')
    for li in lis:
        rslist= {'reple' : li.p.text, 'score': li.find('em').getText()}
        re_list.append(rslist)
        
        
        #watch_movie = li.find('span', {'class':'ico_viewer'})

전처리 조사버리기

In [14]:
def one_movie_prepro(re_list): 
    import numpy as np
    df=pd.DataFrame(re_list)
    df.columns = ["리뷰","평점"]

    df['리뷰']=df['리뷰'].str.replace(pat='관람객', repl='', regex=False)
    df[df['리뷰'].str.contains("스포일러가 포함된 감상평입니다. 감상평 보기")]= np.nan #nan으로 변경

    df['리뷰']= df['리뷰'].str.strip()  # 앞 뒤 공백을 제거
    df=df.replace('',np.NaN)
    df=df.dropna()    
#     df.dropna()
    return df

예매 1순위 가져옴

In [24]:
def one_movie_get_review():
    #1917 영화

    df = pd.read_csv('movie_url_list.csv')
    url_list = df['url'].tolist()

    # for url in url_list :
        #    print(url)
    test_url = url_list[0]
    resp = requests.get(test_url)
    html = BeautifulSoup(resp.content, 'html.parser')
    result = html.find('div', {'class':'score_total'}).find('strong').findChildren('em')[0].getText()
    total_count = int(result.replace(',', ''))
    re_list= []


    for i in range(1, int(total_count / 10) + 1):
    #for i in range(1, int(total_count / 100) + 1):
        url = test_url + '&page=' + str(i)
        #print('url: "' + url + '" is parsing....')
        get_data(url,re_list)
    review_df = pd.DataFrame()
    review_df=one_movie_prepro(re_list)

    return review_df
#     re_list


In [16]:
df=one_movie_get_review()


In [17]:
df.tail(60)

,리뷰,평점
137,로운님과 웬디님 두분 모두 성우가 아님에도 불구하고 정말정말 잘 해주셔서 재밌게 봤...,10
138,코로나 땜에 지루했었는데 트롤덕분에 한시간반이 너무 재밌었어요 !!! 웬디 로운 더...,10
139,"KPOP, Respect.",7
142,"스토리, 연출, 유머, 감동, 색감, 성우들 연기, 노래까지 뭐 하나 빠질게 없는 영화",10
143,다채로운 영상미와 다양한 음악이 나와 신나요! 귀여운팝업북 본 느낌 ㅋㅋ 조금 유치...,10
144,"너무 재밌게 봤어요, 그런데, 최소한 중학생 이상 되야 재미를 느낄 것 같아요. 영...",10
146,재미 있었습니다!! 다시 또 봐야겠어요~~~~,10
149,0 / 11 / 3 신드라 : 15 open gg,10
154,영상미가 아주 나이스입ㄴ다,10
156,Ost가 너무너무좋아요! 더빙성우분들이 너무 잘소화하시구 잘불러서 더빙판ost발매가...,10


In [22]:
def step7_insert_review(df) :
    # 객체를 복원한다.
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)

    import numpy as np
    
    rlist=df["리뷰"].tolist()
    for i in rlist :
        text = i

        str = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(str) * 100)
        # 예측 결과
        r2 = pipe.predict(str)[0]

        if r2 == '1' :
            print(text)
            print('긍정적인 리뷰')
            print('----------------------------')
        else :
            print(text)
            print('부정적인 리뷰')
            print('----------------------------')

        print('정확도 : %.3f' % r1)

In [23]:
step7_insert_review(df)

더빙판 걱정반 기대반 봤는데 생각보다 웬디랑 로운 연기가 자연스럽고 어색하지 않아서 놀랬음. 노래도 기대 이상으로 잘해서 더빙판이 매우 흡족스러움. 아이돌이 연기하면 더군다나 더빙하면 진짜 기대는 커녕 걱정만 하고 ...
긍정적인 리뷰
----------------------------
정확도 : 93.454
트롤들 다 귀여워요ㅠㅠㅠ 그리고 웬디랑 로운 더빙 정말 잘해서 깜짝놀랐어요 !!
긍정적인 리뷰
----------------------------
정확도 : 99.193
더빙으로봤는데 너무좋았어요!!
긍정적인 리뷰
----------------------------
정확도 : 98.344
코로나로 우중충한 분위기를 넘나 경쾌하게 띄워주는 귀요미들.. 극장하고  VOD 동시에 개봉하는 방법도 괜찮은 듯 하다.
긍정적인 리뷰
----------------------------
정확도 : 98.723
진짜 재밌어요 특히 웬디님 목소리가 대박이에요 ㅠㅠ다들 더빙으로 꼭 한번 보고 오세요
긍정적인 리뷰
----------------------------
정확도 : 99.788
트롤들이 너무 졸귀탱이야ㅠㅠ
긍정적인 리뷰
----------------------------
정확도 : 97.065
웬디 노래 넘 잘해요..연기도 자연스럽고..별 기대안했는데 생각보다 정신없이 봤네요..8살은 정말재밌다네요..
긍정적인 리뷰
----------------------------
정확도 : 99.523
오랜만에 본 트롤 아이들도 너무좋아했고 방가웠어요중간중간 나오는 노래도 재밌고 내용또한 재미났어요
긍정적인 리뷰
----------------------------
정확도 : 93.821
ost 짱 좋아요 보다가 케이팝 트롤에 레드벨벳 나와서 반가웠네요ㅋㅋㅋㅋ 가사도 우리말로 나오고!
긍정적인 리뷰
----------------------------
정확도 : 94.838
진짜 무료한 일상중에 신나게 영화 잘봤어요 !! 특히 트롤들이 너무 귀여워서 ㅠ ost도 넘 좋고 중간에 살

아이들도 어른도 감동적인 영화입니다~음악의 역사&종류가 한자리에 또 보고 싶네요. 세아이들은 집으로 와서 다양한 음악성을 논하느라 여념이 없네요.
긍정적인 리뷰
----------------------------
정확도 : 99.506
정말 감동적이였어요 마지막에 모든 트롤종족들이 화해하고 노래 부를때 온 몸에 전율이 쫙 끼치더라구요ㅜ~~~ 팝트롤 여왕이 조금 발암역할이였지만 나머진 모두 괜찮은거 같아용^_^
긍정적인 리뷰
----------------------------
정확도 : 98.636
노래들이 다 너무 좋았고 더빙하신 웬디님 로운님이 너무 잘하셔서 깜짝 놀랐어요
긍정적인 리뷰
----------------------------
정확도 : 99.682
너무 잼있어서 더빙 자막판 두번 봤네요. 신나는 음악에 감동적인 교훈도 있어서 남녀노소 모두다 즐겁게 볼수 있을거 같습니다. 원래는 애니매이션 더빙은 안좋아하는 편이었는데 웬디씨가 하신다고 해서 관심 가지고 봤는데 ...
긍정적인 리뷰
----------------------------
정확도 : 80.859
아 왜 하필 지금시기에 개봉이냐 ㅠㅠ 관객많이와야하는데 퀸짱벨벳 화이팅
긍정적인 리뷰
----------------------------
정확도 : 97.133
로운 연기 궁금해서 본건데 진짜 목소리 좋아요
긍정적인 리뷰
----------------------------
정확도 : 99.549
재밌게 잘 봤어요. 가볍게 보기 좋고 남녀노소 다 좋아할 것 같습니다. 그리고 더빙을 너무 잘해서 많이 놀랐어요.
긍정적인 리뷰
----------------------------
정확도 : 99.502
개재밌음 뭔가 유튜브 병맛영상같은 느낌도 있으면서 언어유희랑 추억노래들 나오는데 아이들보다 성인이 더 즐겁게 볼수도 있다. 극장 나오고 ost전부 찾아서 들어봄
긍정적인 리뷰
----------------------------
정확도 : 93.368
웬디 로룬 연기 굿!
긍정적인 리뷰
-----------

아 ? 감독에 문제가 있는듯 ?영상미는 있는데 내용이 ㅜㅜ졸려요와... 알바들 무섭네요~~ 제 글 비추가 13인데.... 제 글 아래 비추가 1도 없어요.... ㅋㅋ
부정적인 리뷰
----------------------------
정확도 : 81.569
1편 재밌게봐서 나오자마자 16000원 주고 봤는데 이번엔 좀 별로네
긍정적인 리뷰
----------------------------
정확도 : 83.914
지루하고 졸렸음..13세 딸도 별로였다고.넘 유치트롤1편 완전 재밌게 본 엄마와 딸인데 실망.케이팝도 별로 안 나오고스토리도 별로고더구나 음악도 감동도 없었었던
부정적인 리뷰
----------------------------
정확도 : 93.881
방심하지마라 아직 영화관 갈 때 아니다..
긍정적인 리뷰
----------------------------
정확도 : 93.494
진짜 노잼이네유  정신없고 산만하고 뜸금없네여
부정적인 리뷰
----------------------------
정확도 : 91.908
